In [ ]:
"""
Description:
    This script processes mass spectrometry (MS) data to calculate fragment ion information
    and noise factors. It replaces the slower np.isclose-based matching with a faster
    KDTree-based matching algorithm (using SciPy's cKDTree) to efficiently locate m/z values
    within a specified tolerance. The script leverages multiprocessing for parallel processing
    of spectra and uses tqdm to display a progress bar during execution.

Algorithms and Techniques:
    - KDTree-based nearest neighbor search for fast ion matching.
    - Multiprocessing (Pool) to parallelize the processing of spectra.

Author: Ling Luo
Date: 2025-02-26
"""

In [ ]:
import re

def extract_modified_amino_acids(peptide_sequence):
    """
    Extracts modified amino acids from a peptide sequence.
    """
    matches = re.finditer(r'[A-Za-z]\([^)]+\)', peptide_sequence)
    results = [(match.start(), match.group()) for match in matches]
    return results


def split_peptide_sequence(peptide_sequence):
    """
    Splits a peptide sequence into individual amino acids, accounting for any modifications.
    """
    count = 0
    amino_acids = re.findall(r'[A-Z][a-z]*', peptide_sequence)
    extract_modified_amino_acids_lst = extract_modified_amino_acids(peptide_sequence)
    if extract_modified_amino_acids_lst:
        for item in extract_modified_amino_acids_lst:
            idx = item[0] - count
            modified_amino_acids = item[1]
            amino_acids[idx] = modified_amino_acids
            count += len(modified_amino_acids) - 1
    return amino_acids

def modification_replacer(sequence):
    """
    Replaces specific modifications in a peptide sequence with standardized characters.
    """
    seq_list = split_peptide_sequence(sequence)
    result_list = [
        item.replace("C(+57.02)", "C").replace("M(+15.99)", "m").replace("N(+.98)", "n").replace("Q(+.98)", "q") for item in seq_list
    ]
    return ''.join(result_list)

In [ ]:
from tqdm import tqdm

def read_mgf(file_path):
    """
    Reads an MGF (Mascot Generic Format) file and extracts spectra information.
    Each spectrum is represented as a dictionary with metadata and a list of peaks.
    
    Args:
        file_path (str): Path to the MGF file.
        
    Returns:
        list: A list of dictionaries, each containing metadata and peaks for a spectrum.
    """
    spectra = []
    with open(file_path, 'r') as file:
        spectrum = {}
        peaks = []
        in_spectrum = False

        for line in file:
            line = line.strip()
            
            if line == "BEGIN IONS":
                # Start of a new spectrum
                spectrum = {}
                peaks = []
                in_spectrum = True
            elif line == "END IONS":
                # End of the current spectrum
                spectrum["peaks"] = peaks
                spectra.append(spectrum)
                in_spectrum = False
            elif in_spectrum:
                # Parse metadata or peaks if within a spectrum
                if "=" in line:
                    key, value = line.split("=", 1)
                    spectrum[key.strip().lower()] = value.strip()
                else:
                    match = re.match(r"([\d.]+)\s+([\d.]+)", line)
                    if match:
                        mass_charge_ratio = float(match.group(1))
                        intensity = float(match.group(2))
                        peaks.append((mass_charge_ratio, intensity))
    return spectra


spec = read_mgf('antibody-gw-finalData/131Ab_20250213_DBsearch_reformed_HCD_only3mod.mgf')
seq_all = [spec[i]['seq'] for i in range(len(spec))]
seq_all_modified = [modification_replacer(item) for item in seq_all]

In [3]:
from pyteomics import mgf, mass

# mass value
mass_H = 1.0078
mass_H2O = 18.0106
mass_NH3 = 17.0265
mass_N_terminus = 1.0078
mass_C_terminus = 17.0027
mass_CO = 27.9949
mass_Phosphorylation = 79.96633

mass_AA = {
    "_PAD": 0.0,
    "_GO": mass_N_terminus - mass_H,
    "_EOS": mass_C_terminus + mass_H,
    "A": 71.03711,  # 0
    "R": 156.10111,  # 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    "N": 114.04293,  # 2
    "n": 115.02695,
    "D": 115.02694,  # 3
    "C": 160.03065,  # 103.00919,  # 4
    "E": 129.04259,  # 5
    "Q": 128.05858,  # 6
    "q": 129.0426,
    "G": 57.02146,  # 7
    "H": 137.05891,  # 8
    "I": 113.08406,  # 9
    "L": 113.08406,  # 10
    "K": 128.09496,  # 11
    "M": 131.04049,  # 12
    "m": 147.0354,
    "F": 147.06841,  # 13
    "P": 97.05276,  # 14
    "S": 87.03203,  # 15
    "T": 101.04768,  # 16
    "W": 186.07931,  # 17
    "Y": 163.06333,  # 18
    "V": 99.06841,  # 19
}

In [4]:
def fragments(peptide, types=('b', 'a', 'y')):
    """
    The function generates possible m/z for fragments of 8 ion types
    b, b2+, b-NH3, b-H2O, y, y2+, y-NH3, y-H2O
        :param
            peptide: peptide sequence as string
            types: types of fragment ions f.e. (b,y) or (a,x)
        :return
            b_ions: list of list of b_ions
            y_ions: list of list of y_ions
    """
    a_ions = []
    b_ions = []
    y_ions = []

    for i in range(1, len(peptide)):
        for ion_type in types:
            if ion_type[0] in 'b':
                b_ion_types = []
                # b
                b_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA))
                # b(2+)
                b_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=2, aa_mass=mass_AA))
                # b-H2O
                b_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='H2O', charge=1))
                # b-NH3
                b_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='NH3', charge=1))
                b_ions.append(b_ion_types)
            if ion_type[0] in 'a':
                a_ion_types = []
                # a
                a_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA))
                # a(2+)
                a_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=2, aa_mass=mass_AA))
                # a-H2O
                a_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='H2O', charge=1))
                # a-NH3
                a_ion_types.append(mass.fast_mass(
                        peptide[:i], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='NH3', charge=1))
                a_ions.append(a_ion_types)
            else:
                y_ion_types = []
                # y
                y_ion_types.append(mass.fast_mass(
                        peptide[i:], ion_type=ion_type, charge=1, aa_mass=mass_AA))
                # y(2+)
                y_ion_types.append(mass.fast_mass(
                        peptide[i:], ion_type=ion_type, charge=2, aa_mass=mass_AA))
                # y-H2O
                y_ion_types.append(mass.fast_mass(
                        peptide[i:], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='H2O', charge=1)) # if its double charged is it only half the mass
                # y-NH3
                y_ion_types.append(mass.fast_mass(
                        peptide[i:], ion_type=ion_type, charge=1, aa_mass=mass_AA) - mass.calculate_mass(formula='NH3', charge=1))
                y_ions.append(y_ion_types)
    return a_ions, b_ions, y_ions

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from multiprocessing import Pool
import logging

# 配置日志输出
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

match_mass_tol = 0.1  # in Da

def process_spectrum(args):
    """
    Processes a single spectrum using KDTree-based matching.
    Replaces np.isclose by quickly searching for candidate m/z values within the matching tolerance.
    
    Returns:
        A tuple containing:
            - missing_cleavage: number of missing cleavage sites
            - cleavage_position: list of present cleavage positions
            - missing_cleavage_including_aions: missing cleavage count including a-ions
            - cleavage_position_including_aions: list of cleavage positions including a-ions
            - intensity_array: modified intensity array with fragment peaks set to NaN
    """
    spectrum, peptide, tol = args
    # 如果肽段无效（如为NaN），直接返回缺失值，并保持强度数组不变
    if isinstance(peptide, float):
        return (np.nan, np.nan, np.nan, np.nan, spectrum['intensity array'])
    
    # 获取理论碎片离子：a、b、y离子列表
    a_ions, b_ions, y_ions = fragments(peptide)
    
    missing_cleavage = 0
    cleavage_position = []
    missing_cleavage_including_aions = 0
    cleavage_position_including_aions = []
    
    mz_array = spectrum['m/z array']
    # 复制一份强度数组，后续会将匹配到的离子强度设为NaN
    intensity_array = spectrum['intensity array'].copy()
    # 构建1维KDTree（注意必须将数组reshape为(n,1)）
    tree = cKDTree(mz_array.reshape(-1, 1))
    
    # 遍历每个裂解位点的离子（注意：y离子列表采用反向遍历，与原代码一致）
    for pos, (a_frag, b_frag, y_frag) in enumerate(zip(a_ions, b_ions, reversed(y_ions))):
        # 使用KDTree快速查询，每个离子候选值在谱图中是否存在（在tol范围内）
        a_present = any(len(tree.query_ball_point(i, r=tol)) > 0 for i in a_frag)
        b_present = any(len(tree.query_ball_point(i, r=tol)) > 0 for i in b_frag)
        y_present = any(len(tree.query_ball_point(i, r=tol)) > 0 for i in y_frag)
        
        # 判断是否检测到b或y离子（对应常规裂解）
        if b_present or y_present:
            cleavage_position.append(pos + 1)
        else:
            missing_cleavage += 1
        
        # 判断是否检测到a、b或y离子（包含a离子）
        if b_present or y_present or a_present:
            cleavage_position_including_aions.append(pos + 1)
        else:
            missing_cleavage_including_aions += 1

    # 对于b和y离子，再次遍历并将匹配到的峰在强度数组中置为NaN
    for b_frag, y_frag in zip(b_ions, reversed(y_ions)):
        for candidate in b_frag:
            indices = tree.query_ball_point(candidate, r=tol)
            for idx in indices:
                intensity_array[idx] = np.nan
        for candidate in y_frag:
            indices = tree.query_ball_point(candidate, r=tol)
            for idx in indices:
                intensity_array[idx] = np.nan

    return (missing_cleavage, cleavage_position,
            missing_cleavage_including_aions, cleavage_position_including_aions,
            intensity_array)

def noise_and_fragmentIons(mgf_in_path):
    """
    Processes an MGF file to calculate fragment ion matching statistics using KDTree-based matching.
    Uses multiprocessing to parallelize the processing of spectra.

    Returns:
        df: DataFrame with additional columns for missing cleavages and cleavage positions.
        median_noise: The median noise intensity across all spectra.
    """
    df = pd.DataFrame()
    missing_cleavages = []
    cleavage_positions = []
    missing_cleavages_including_aions = []
    cleavage_positions_including_aions = []
    noise_intensities_all = []
    
    tasks = []
    # 利用mgf.read依次读取每个谱图，与对应的肽段（seq_all_modified）组合成任务
    with mgf.read(mgf_in_path) as spectra:
        for spectrum, peptide in zip(spectra, seq_all_modified):
            tasks.append((spectrum, peptide, match_mass_tol))
    
    # 使用多进程并行处理所有谱图，并显示进度条
    with Pool() as pool:
        results = []
        for res in tqdm(pool.imap(process_spectrum, tasks), total=len(tasks), desc="Processing Spectra"):
            results.append(res)
    
    # 汇总每个谱图返回的结果
    for res in results:
        missing_cleavage, cleavage_pos, missing_cleavage_including, cleavage_pos_including, mod_intensity = res
        missing_cleavages.append(missing_cleavage)
        cleavage_positions.append(cleavage_pos)
        missing_cleavages_including_aions.append(missing_cleavage_including)
        cleavage_positions_including_aions.append(cleavage_pos_including)
        noise_intensities_all.extend(mod_intensity.tolist())
    
    df["Number of missing cleavages"] = missing_cleavages
    df["Position of present cleavages"] = cleavage_positions
    df["Number of missing cleavages (including a-ions)"] = missing_cleavages_including_aions
    df["Position of present cleavages (including a-ions)"] = cleavage_positions_including_aions
    
    median_noise = np.nanmedian(noise_intensities_all)
    median_missing_cleavages = np.nanmedian(missing_cleavages)
    logger.info(f"Median noise intensity: {median_noise}")
    logger.info(f"Median number of missing cleavages: {median_missing_cleavages}")
    
    return df, median_noise

if __name__ == "__main__":
    mgf_in_path = "antibody-gw-finalData/131Ab_20250213_DBsearch_reformed_HCD_only3mod.mgf"
    df, median_noise = noise_and_fragmentIons(mgf_in_path)
    print(df.head())
    print("Median noise intensity:", median_noise)


Processing Spectra: 100%|██████████| 1638248/1638248 [05:46<00:00, 4722.71it/s]
INFO:__main__:Median noise intensity: 3216.0732
INFO:__main__:Median number of missing cleavages: 0.0


   Number of missing cleavages Position of present cleavages  \
0                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
1                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
2                            0            [1, 2, 3, 4, 5, 6]   
3                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
4                            0      [1, 2, 3, 4, 5, 6, 7, 8]   

   Number of missing cleavages (including a-ions)  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                                               0   

  Position of present cleavages (including a-ions)  
0                         [1, 2, 3, 4, 5, 6, 7, 8]  
1                         [1, 2, 3, 4, 5, 6, 7, 8]  
2                               [1, 2, 3, 4, 5, 6]  
3                         [1, 2, 3, 4, 5, 6, 7, 8]  
4                         [1, 2

In [8]:
df.to_csv('../../data/code/miss_and_noise.csv')

In [12]:
df["Modified Sequence"] = seq_all_modified
df

,Number of missing cleavages,Position of present cleavages,Number of missing cleavages (including a-ions),Position of present cleavages (including a-ions),Modified Sequence
0,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC
1,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC
2,0,"[1, 2, 3, 4, 5, 6]",0,"[1, 2, 3, 4, 5, 6]",KVEPKSC
3,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC
4,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC
...,...,...,...,...,...
1638243,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17...",7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17...",ITDFFPEDITVEWQWNGQPAENYK
1638244,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",LLIYFASTR
1638245,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",DDPEVQFSWFVDDVEVHTAQTQPR
1638246,1,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]",1,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]",QNGVLNSWTDQDSK


In [13]:
number_of_miss_includ_a = df['Number of missing cleavages (including a-ions)'].tolist()
missing_at_least_one_fragment = 0

for item in number_of_miss_includ_a:
    if item > 0:
        missing_at_least_one_fragment += 1
missing_at_least_one_fragment/len(number_of_miss_includ_a)

0.44075927454207176

In [14]:
import numpy as np
from scipy.spatial import cKDTree
import logging
from pyteomics import mgf

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def noise_factor(df, mgf_in_path, median_noise):
    """
    Adds and calculates the noise factor to a summary DataFrame.

    :param df: Summary DataFrame which should include a column "Modified Sequence"
    :param mgf_in_path: Path to the MGF file
    :param median_noise: The median noise value for the entire dataset, used to classify noise peaks
    :return: DataFrame with additional columns: 'Noise factor', 'Number of noise peaks', 'Number of fragment peaks'
    """
    noise_factor_list = [] 
    noise_total_amount = []
    fragments_total_amount = []

    with mgf.read(mgf_in_path) as spectra:
        # Convert spectra to a list to allow progress tracking
        spectra = list(spectra)
        if len(spectra) != len(df):
            logger.error("The mgf file and summary file have different numbers of spectra. Check if you chose the correct corresponding files")
            return df 
        # Iterate over spectra with a progress bar
        for spectrum, peptide in tqdm(zip(spectra, df["Modified Sequence"]), total=len(spectra), desc="Processing Noise"):
            if not isinstance(peptide, float):
                amount_noise_peaks = 0
                amount_fragment_peaks = 0
                a_ions, b_ions, y_ions = fragments(peptide)
                # Make a copy of the intensity array; matched peaks will be set to NaN
                intensity_array = spectrum['intensity array'].copy()
                mz_array = spectrum['m/z array']
                # Build a 1D KDTree (reshape the array to 2D)
                tree = cKDTree(mz_array.reshape(-1, 1))
                
                # For each cleavage site, match b and y ions (y ions are processed in reverse order to maintain original behavior)
                for b_ion, y_ion in zip(b_ions, reversed(y_ions)):
                    # Process b ions
                    for candidate in b_ion:
                        indices = tree.query_ball_point([candidate], r=match_mass_tol)
                        if indices:
                            intensity_array[indices] = np.nan
                            amount_fragment_peaks += len(indices)
                    # Process y ions
                    for candidate in y_ion:
                        indices = tree.query_ball_point([candidate], r=match_mass_tol)
                        if indices:
                            intensity_array[indices] = np.nan
                            amount_fragment_peaks += len(indices)
                
                # Count noise peaks: remaining peaks with intensity >= median_noise are considered noise peaks
                for peak in intensity_array:
                    if peak >= median_noise:
                        amount_noise_peaks += 1

                noise_total_amount.append(amount_noise_peaks)
                fragments_total_amount.append(amount_fragment_peaks)
                if amount_fragment_peaks > 0:
                    noise_val = amount_noise_peaks / amount_fragment_peaks
                else:
                    noise_val = amount_noise_peaks  # Avoid division by zero
                noise_factor_list.append(noise_val)
            else:
                noise_factor_list.append(np.nan)
                fragments_total_amount.append(np.nan)
                noise_total_amount.append(np.nan)
                
        df["Noise factor"] = noise_factor_list
        df["Number of noise peaks"] = noise_total_amount
        df["Number of fragment peaks"] = fragments_total_amount

        total_peaks = np.nansum(noise_total_amount) + np.nansum(fragments_total_amount)
        noise_percent = np.nansum(noise_total_amount) / total_peaks
        logger.info(f"{noise_percent*100:.2f}% of all peaks are noise")
        logger.info(f"Average noise factor: {np.nanmean(noise_factor_list)}")
    return df

if __name__ == "__main__":
    df_noise = noise_factor(df, mgf_in_path, median_noise)
    print(df.head())


Processing Noise: 100%|██████████| 1638248/1638248 [1:29:11<00:00, 306.13it/s]
INFO:__main__:76.56% of all peaks are noise
INFO:__main__:Average noise factor: 3.092431088762153


   Number of missing cleavages Position of present cleavages  \
0                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
1                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
2                            0            [1, 2, 3, 4, 5, 6]   
3                            0      [1, 2, 3, 4, 5, 6, 7, 8]   
4                            0      [1, 2, 3, 4, 5, 6, 7, 8]   

   Number of missing cleavages (including a-ions)  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                                               0   

  Position of present cleavages (including a-ions) Modified Sequence  \
0                         [1, 2, 3, 4, 5, 6, 7, 8]         DKKVEPKSC   
1                         [1, 2, 3, 4, 5, 6, 7, 8]         DKKVEPKSC   
2                               [1, 2, 3, 4, 5, 6]           KVEPKSC   
3       

In [15]:
df_noise.to_csv('../../data/code/noise.csv')

In [16]:
df_noise

,Number of missing cleavages,Position of present cleavages,Number of missing cleavages (including a-ions),Position of present cleavages (including a-ions),Modified Sequence,Noise factor,Number of noise peaks,Number of fragment peaks
0,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC,1.387755,68,49
1,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC,0.730769,19,26
2,0,"[1, 2, 3, 4, 5, 6]",0,"[1, 2, 3, 4, 5, 6]",KVEPKSC,1.000000,16,16
3,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC,6.146341,252,41
4,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",DKKVEPKSC,4.485714,157,35
...,...,...,...,...,...,...,...,...
1638243,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17...",7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17...",ITDFFPEDITVEWQWNGQPAENYK,1.608696,37,23
1638244,0,"[1, 2, 3, 4, 5, 6, 7, 8]",0,"[1, 2, 3, 4, 5, 6, 7, 8]",LLIYFASTR,7.782609,179,23
1638245,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",DDPEVQFSWFVDDVEVHTAQTQPR,1.792453,95,53
1638246,1,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]",1,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]",QNGVLNSWTDQDSK,2.218750,71,32
